In [1]:
import requests
import json
import pandas as pd

In [19]:
# Assume the JSON data is stored in a variable called `json_data`
data = json.loads(response.text)

In [22]:
# Extract the reviews from the JSON data and convert it to a DataFrame
reviews = pd.DataFrame(data['reviews'])

In [25]:
reviews.to_csv("reviews_sample.csv",index=False)

In [3]:
asin_list_path = '/Users/vladbordei/Documents/Development/ProductExplorer/data/external/asin_list.csv'
asin_list = pd.read_csv(asin_list_path)['asin'].tolist()

In [4]:
asin_list

['B08X2324ZL',
 'B09MQ689XL',
 'B0BCW8X98V',
 'B09NR5JW8Y',
 'B0BHS5VPCZ',
 'B0BCVY1DHR',
 'B0BNXCJ3MV',
 'B0BCTTCSBZ',
 'B091325ZMB',
 'B093HCXGC9',
 'B0BL6MR1KN',
 'B09PN11Y65',
 'B0BLNBS36G',
 'B0BVZ3H4MB',
 'B0BW8Y2B8Z',
 'B0BDFZPNVY']

In [ ]:
# Loop over ASINs
for asin in asin_list:
    details = get_product_details(asin)
    reviews = get_product_reviews(asin)
    update_firestore(asin, details, reviews)

In [13]:
import time
import asyncio
import nest_asyncio
import aiohttp
import json
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore






# Your ASINs
asin_list = ['B08X2324ZL', 'B09MQ689XL']

# Amazon Scraper details
base_url = "https://asin-data.p.rapidapi.com/request"
api_key = "70201ee0c8ed29661bc6ae00a84341fb"
headers = {
	"X-RapidAPI-Key": "4da31a08e5mshaca05d98a3d9d6ep1fffb1jsn019717508cc8",
	"X-RapidAPI-Host": "asin-data.p.rapidapi.com"
}

# Firestore details
cred_path = '/Users/vladbordei/Documents/Development/ProductExplorer/notebooks/productexplorerdata-firebase-adminsdk-ulb3d-465f23dff3.json'

# Initialize Firestore
cred = credentials.Certificate(cred_path)
if not firebase_admin._apps:
    firebase_admin.initialize_app(cred)
db = firestore.client()

async def get_product_details(asin):
    start = time.time()
    async with aiohttp.ClientSession() as session:
        url = f"{base_url}{asin}"
        async with session.get(url, headers=headers, params={"api_key": api_key}) as response:
            if response.status != 200:
                print(f"Failed to fetch product details for {asin}. HTTP status: {response.status}")
                return None
            print(f"Fetching product details for {asin} took {time.time() - start} seconds.")
            return await response.json()

async def fetch(session, page_var, asin):
    params = {
        "type":"reviews",
        "asin":asin,
        "amazon_domain":"amazon.com",
        "reviewer_type":"verified_purchase",
        "sort_by":"most_recent",
        "global_reviews":"false",
        "page":page_var
    }
    async with session.get(url=base_url, headers=headers, params=params) as response:
        if response.status != 200:
            print(f"Failed to fetch reviews for {asin} page {page_var}. HTTP status: {response.status}")
            return None
        return await response.json()

async def get_product_reviews(asin):
    start = time.time()
    async with aiohttp.ClientSession() as session:
        pages = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10"]
        tasks = [fetch(session, page_var, asin) for page_var in pages]
        results = await asyncio.gather(*tasks)
        print(f"Fetching product reviews for {asin} took {time.time() - start} seconds.")
        return results

def update_firestore(asin, details, reviews):
    if details is None or reviews is None or any(review is None for review in reviews):
        print(f"Skipping Firestore update for {asin} due to missing data.")
        return
    start = time.time()
    doc_ref = db.collection('products').document(asin)
    doc_ref.set({
        'details': details,
        'reviews': reviews
    }, merge=True)
    print(f"Updating Firestore for {asin} took {time.time() - start} seconds.")

async def process_asin(asin):
    details = await get_product_details(asin)
    if details is None:
        print(f"Skipping {asin} due to failed details fetch.")
        return
    reviews = await get_product_reviews(asin)
    if reviews is None or any(review is None for review in reviews):
        print(f"Skipping {asin} due to failed reviews fetch.")
        return
    update_firestore(asin, details, reviews)

async def main():
    tasks = [process_asin(asin) for asin in asin_list]
    await asyncio.gather(*tasks)

loop = asyncio.get_event_loop()
loop.run_until_complete(main())


Failed to fetch product details for B08X2324ZL. HTTP status: 404
Skipping B08X2324ZL due to failed details fetch.
Failed to fetch product details for B09MQ689XL. HTTP status: 404
Skipping B09MQ689XL due to failed details fetch.


In [11]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

# Firestore details
cred_path = '/Users/vladbordei/Documents/Development/ProductExplorer/notebooks/productexplorerdata-firebase-adminsdk-ulb3d-465f23dff3.json'

# Initialize Firestore
cred = credentials.Certificate(cred_path)
if not firebase_admin._apps:
    firebase_admin.initialize_app(cred)
db = firestore.client()

# ASIN
asin = 'B09MQ689XL'  # Replace with the ASIN you are interested in

# Get the document with the provided ASIN
doc_ref = db.collection('products').document(asin)

# Get the document data
doc = doc_ref.get()

# Check if document exists
if doc.exists:
    # Get reviews
    reviews = doc.to_dict().get('reviews', {})
    print(reviews)
else:
    print(f"No such document for ASIN: {asin}")


[{'message': 'API Request failed due to Provider configuration error: The API response body transformation failed to parse content-type application/json. Please contact the API Provider', 'info': 'Your Client (working) ---> Gateway (working) ---> API (not working)'}, {'message': 'API Request failed due to Provider configuration error: The API response body transformation failed to parse content-type application/json. Please contact the API Provider', 'info': 'Your Client (working) ---> Gateway (working) ---> API (not working)'}, {'message': 'API Request failed due to Provider configuration error: The API response body transformation failed to parse content-type application/json. Please contact the API Provider', 'info': 'Your Client (working) ---> Gateway (working) ---> API (not working)'}, {'message': 'API Request failed due to Provider configuration error: The API response body transformation failed to parse content-type application/json. Please contact the API Provider', 'info': 'Yo

In [12]:
len(reviews['reviews'])

TypeError: list indices must be integers or slices, not str

In [4]:
reviews.keys()

dict_keys(['2_star_ratings', '3_star_percentage', 'average_rating', '1_star_ratings', '5_star_percentage', '3_star_ratings', 'top_critical_review', 'top_positive_review', 'product', 'total_ratings', '5_star_ratings', 'pagination', '4_star_percentage', '2_star_percentage', '4_star_ratings', '1_star_percentage', 'filtered_total_ratings', 'reviews', 'message', 'total_reviews', 'filtered_total_reviews'])

In [6]:
reviews[ 'pagination'].keys()

AttributeError: 'list' object has no attribute 'keys'

In [7]:
reviews[ 'pagination']

['https://www.amazon.com/ap/signin?openid.return_to=https%3A%2F%2Fwww.amazon.com%2FKING-BIRD-Galvanized-Heightened-Deep-Rooted%2Fproduct-reviews%2FB09MQ689XL%3Fie%3DUTF8%26pageNumber%3D2&openid.identity=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.assoc_handle=usflex&openid.mode=checkid_setup&marketPlaceId=ATVPDKIKX0DER&language=en&openid.claimed_id=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.ns=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0']

In [29]:
reviews[ 'filtered_total_reviews']

583

In [ ]:
API Request failed due to Provider configuration error: The API response body transformation failed to parse content-type application/json. Please contact the API Provider